# 0. Tools

#### permutation

In [1]:
from itertools import permutations

# 3!
for perm in permutations([1,2,3]):
    print(perm)

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


#### combination

In [2]:
from itertools import combinations

# 4C2
for comb in combinations([1,2,3,4], 2):
    print(comb)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


#### product

In [3]:
from itertools import product

# 2*2*3
for x1, x2, x3 in product([1,2], ['a', 'b'], ['xx', 'yy', 'zz']):
    print(x1, x2, x3)

1 a xx
1 a yy
1 a zz
1 b xx
1 b yy
1 b zz
2 a xx
2 a yy
2 a zz
2 b xx
2 b yy
2 b zz


In [4]:
from itertools import product

# 2*2*2*2
for x1, x2, x3, x4 in product([1,2], repeat=4):
    print(x1, x2, x3, x4)

1 1 1 1
1 1 1 2
1 1 2 1
1 1 2 2
1 2 1 1
1 2 1 2
1 2 2 1
1 2 2 2
2 1 1 1
2 1 1 2
2 1 2 1
2 1 2 2
2 2 1 1
2 2 1 2
2 2 2 1
2 2 2 2


#### SAT-solver
[https://pypi.org/project/pycosat/](https://pypi.org/project/pycosat/)

- Example 1: Solve the SAT
    + ($x_1 = 0$ or $x_2 = 1$)($x_1 = 1$ or $x_2 = 1$)($x_1 = 1$ or $x_2 = 0$)
    + $(\bar{x_1} + x_2)(x_1 + x_2)(x_1 + \bar{x_2})$

In [5]:
from pycosat import solve

# Solution: x1 = 1, x2 = 1
print(solve([[-1, 2], [1,2], [1, -2]]))

[1, 2]


- Example 2: Solve the SAT
    + ($x_1 = 1$ or $x_2 = 0$ or $x_3 = 0$)($x_3 = 1$ or $x_1 = 0$)($x_2 = 1$ or $x_3 = 1$)
    + $(x_1 + \bar{x_2} + \bar{x_3})(x_3 + \bar{x_1})(x_2 + x_3)$

In [6]:
# Solution: x1=0, x2=1, x3=0
print(solve([[1, -2, -3], [3, -1], [2, 3]]))

[-1, 2, -3]


- Example 3: Solve the SAT
    + ($x_2 = 0$ or $x_3 = 1$)($x_2=1$ or $x_1=1$)($x_3=0$)($x_3=1$ or $x_1=0$)
    + $(\bar{x_2} + x_3)(x_2 + x_1)(\bar{x_3})(x_3 + \bar{x_1})$

In [7]:
print(solve([[-2, 3], [2, 1], [-3], [3, -1]]))

UNSAT


# 1. Bruteforce
#### Models
- Chessboard: N*N
- Represent the all Queen(q0,q1, ...,qN-1) positions by an array A
    + qi at (i,j)
    + `index = i`: col
    + `A[index] = j`: row

<img src="assets/1.png" width="600"/>

#### Constraints
- If array A is a permutation of N (permute(0,1,...N_1)) then N queens will not attack each other by row, col
- Only has to check if each pair of queen is on the same diag
    + Consider 2 queens: `q1 = (i1, j1)`, `q2 = (i2, j2)`
    + q1, q2 are on the same diag if the horizontal shift is the same as the vertical shift: $|i_1 -i_2| = |j_1 - j_2|$

<img src="assets/2.png" width="200"/>


In [8]:
def is_same_diag(i1, j1, i2, j2):
    '''
    q1 = (i1, j1)
    q2 = (i2, j2)
    is in the same diag if abs(i1-i2) = abs(j1-j2)  
    '''
    return abs(i1-i2) == abs(j1-j2)

def is_solution(A):
    '''Check if a permuted config (array A) has all pairs not on the same diag - O(N C2)'''
    # Generate all pairs of q1, q2
    pairs = combinations(range(len(A)), 2)

    # q1 = (i1, A[i1]), q2 = (i2, A[i2])
    return all(not is_same_diag(i1, A[i1], i2, A[i2]) for i1, i2 in pairs)

In [9]:
%%time
# Solve N = 4
N = 4
for perm in permutations(range(N)):
    if is_solution(perm):
        print(perm)

(1, 3, 0, 2)
(2, 0, 3, 1)
CPU times: user 78 µs, sys: 4 µs, total: 82 µs
Wall time: 75.6 µs


In [10]:
%%time
# Solve N = 8
N = 8
for perm in permutations(range(N)):
    if is_solution(perm):
        print(perm)

(0, 4, 7, 5, 2, 6, 1, 3)
(0, 5, 7, 2, 6, 3, 1, 4)
(0, 6, 3, 5, 7, 1, 4, 2)
(0, 6, 4, 7, 1, 3, 5, 2)
(1, 3, 5, 7, 2, 0, 6, 4)
(1, 4, 6, 0, 2, 7, 5, 3)
(1, 4, 6, 3, 0, 7, 5, 2)
(1, 5, 0, 6, 3, 7, 2, 4)
(1, 5, 7, 2, 0, 3, 6, 4)
(1, 6, 2, 5, 7, 4, 0, 3)
(1, 6, 4, 7, 0, 3, 5, 2)
(1, 7, 5, 0, 2, 4, 6, 3)
(2, 0, 6, 4, 7, 1, 3, 5)
(2, 4, 1, 7, 0, 6, 3, 5)
(2, 4, 1, 7, 5, 3, 6, 0)
(2, 4, 6, 0, 3, 1, 7, 5)
(2, 4, 7, 3, 0, 6, 1, 5)
(2, 5, 1, 4, 7, 0, 6, 3)
(2, 5, 1, 6, 0, 3, 7, 4)
(2, 5, 1, 6, 4, 0, 7, 3)
(2, 5, 3, 0, 7, 4, 6, 1)
(2, 5, 3, 1, 7, 4, 6, 0)
(2, 5, 7, 0, 3, 6, 4, 1)
(2, 5, 7, 0, 4, 6, 1, 3)
(2, 5, 7, 1, 3, 0, 6, 4)
(2, 6, 1, 7, 4, 0, 3, 5)
(2, 6, 1, 7, 5, 3, 0, 4)
(2, 7, 3, 6, 0, 5, 1, 4)
(3, 0, 4, 7, 1, 6, 2, 5)
(3, 0, 4, 7, 5, 2, 6, 1)
(3, 1, 4, 7, 5, 0, 2, 6)
(3, 1, 6, 2, 5, 7, 0, 4)
(3, 1, 6, 2, 5, 7, 4, 0)
(3, 1, 6, 4, 0, 7, 5, 2)
(3, 1, 7, 4, 6, 0, 2, 5)
(3, 1, 7, 5, 0, 2, 4, 6)
(3, 5, 0, 4, 1, 7, 2, 6)
(3, 5, 7, 1, 6, 0, 2, 4)
(3, 5, 7, 2, 0, 6, 4, 1)
(3, 6, 0, 7, 4, 1, 5, 2)


# 2. Backtrack
#### Ideas backtrack based on permutation generating

In [11]:
def extend(perm, n):
    if len(perm) == n:
        print(perm)
        return

    for k in range(n):
        if k not in perm:
            extend(perm + [k], n)

In [12]:
extend(perm=[], n=3)

[0, 1, 2]
[0, 2, 1]
[1, 0, 2]
[1, 2, 0]
[2, 0, 1]
[2, 1, 0]


#### Optimized
- Now check only the last added queen (`k`) has the same diag with all prev queen (`[0,k-1]`)
- Time Complexity reduce from `NC2` to `N`

In [13]:
def is_same_diag(i1, j1, i2, j2):
    '''
    q1 = (i1, j1)
    q2 = (i2, j2)
    is in the same diag if abs(i1-i2) = abs(j1-j2)
    '''
    return abs(i1-i2) == abs(j1-j2)

def can_be_extended(A):
    '''Check if the last added queen (k) has not the same diag with the previous [0,k-1] - O(N)'''
    i1 = len(A) - 1
    return all(not is_same_diag(i1, A[i1], i2, A[i2]) for i2 in range(i1))

def extend(perm, n):
    if len(perm) == n:
        print(perm)
        return

    for k in range(n):
        if k not in perm:
            if can_be_extended(perm + [k]):
                extend(perm + [k], n)

In [14]:
%%time
# Solve N = 4
extend(perm=[], n=4)

[1, 3, 0, 2]
[2, 0, 3, 1]
CPU times: user 93 µs, sys: 0 ns, total: 93 µs
Wall time: 80.1 µs


In [15]:
%%time
# Solve N = 8
extend(perm=[], n=8)

[0, 4, 7, 5, 2, 6, 1, 3]
[0, 5, 7, 2, 6, 3, 1, 4]
[0, 6, 3, 5, 7, 1, 4, 2]
[0, 6, 4, 7, 1, 3, 5, 2]
[1, 3, 5, 7, 2, 0, 6, 4]
[1, 4, 6, 0, 2, 7, 5, 3]
[1, 4, 6, 3, 0, 7, 5, 2]
[1, 5, 0, 6, 3, 7, 2, 4]
[1, 5, 7, 2, 0, 3, 6, 4]
[1, 6, 2, 5, 7, 4, 0, 3]
[1, 6, 4, 7, 0, 3, 5, 2]
[1, 7, 5, 0, 2, 4, 6, 3]
[2, 0, 6, 4, 7, 1, 3, 5]
[2, 4, 1, 7, 0, 6, 3, 5]
[2, 4, 1, 7, 5, 3, 6, 0]
[2, 4, 6, 0, 3, 1, 7, 5]
[2, 4, 7, 3, 0, 6, 1, 5]
[2, 5, 1, 4, 7, 0, 6, 3]
[2, 5, 1, 6, 0, 3, 7, 4]
[2, 5, 1, 6, 4, 0, 7, 3]
[2, 5, 3, 0, 7, 4, 6, 1]
[2, 5, 3, 1, 7, 4, 6, 0]
[2, 5, 7, 0, 3, 6, 4, 1]
[2, 5, 7, 0, 4, 6, 1, 3]
[2, 5, 7, 1, 3, 0, 6, 4]
[2, 6, 1, 7, 4, 0, 3, 5]
[2, 6, 1, 7, 5, 3, 0, 4]
[2, 7, 3, 6, 0, 5, 1, 4]
[3, 0, 4, 7, 1, 6, 2, 5]
[3, 0, 4, 7, 5, 2, 6, 1]
[3, 1, 4, 7, 5, 0, 2, 6]
[3, 1, 6, 2, 5, 7, 0, 4]
[3, 1, 6, 2, 5, 7, 4, 0]
[3, 1, 6, 4, 0, 7, 5, 2]
[3, 1, 7, 4, 6, 0, 2, 5]
[3, 1, 7, 5, 0, 2, 4, 6]
[3, 5, 0, 4, 1, 7, 2, 6]
[3, 5, 7, 1, 6, 0, 2, 4]
[3, 5, 7, 2, 0, 6, 4, 1]
[3, 6, 0, 7, 4, 1, 5, 2]


# 3. SAT-solver
#### Setup
- We use $N^2$ variables: $x_{ij}, \forall i,j$ in range `[0,N-1]` 
- $x_{ij} = 1$ iff (if and only if) a queen is placed into the cell (i, j)


#### Constraints
- Each row must have 1 queen: For each $0 \leq i < N$, the i-th row contains at least one queen
    + For all $0 \leq i < N$: $(x_{i1} = 1$ or $x_{i2} = 1$ or $\dots$ or $x_{i(N−1)} = 1)$
- Row constraints: For each $0 \leq i < N$, the i-th row contains at most one queen
    + for all $0 \leq j_1 \neq j_2 < N$: $(x_{ij_1} = 0$ or $x_{ij_2} = 0)$

- Column constraints: For each $0 \leq j < N$, the j-th col contains at most one queen
    + for all $0 \leq i_ 1 \neq i_2 < N$: $(x_{i_1j} = 0$ or $x_{i_2j} = 0)$

- Diag constraints: no two queens stay on the same diagonal
    + For all pair of cell $(i_1, j_1)$ - $(i_2, j_2)$, $i_1 \neq i_2$  if $|i_1 − i_2 | = | j_1 − j_2 |$: $(x_{i_1j_1} = 0$ or $x_{i_2j_2} = 0)$

In [16]:
from itertools import combinations, product
from pycosat import solve

class NQueens:
    def __init__(self, N):
        self.N = N

    def varnum(self, i, j):
        '''Convert (i,j) i,j in range [0,N-1] to varnum(x_num) in range [1, N^2]'''
        assert i in range(self.N) and j in range(self.N)
        return i * self.N + j + 1

    def solve_board(self):
        ## Add constraints
        clauses = []
        # Each row contains at least one queen
        for i in range(self.N):
            clauses.append([self.varnum(i, j) for j in range(self.N)])

        # Row constraints: Each row contains at most one queen
        for i in range(self.N):
            for j1, j2 in combinations(range(self.N), 2):
                clauses.append( [-self.varnum(i, j1), -self.varnum(i, j2)] )

        # Column constraints: each column contains at most one queen
        for j in range(self.N):
            for i1, i2 in combinations(range(self.N), 2):
                clauses.append( [-self.varnum(i1, j), -self.varnum(i2, j)] )

        # Diag constraints: no two queens stay on the same diagonal
        for i1, j1, i2, j2 in product(range(self.N), repeat=4):
            if i1 != i2 and abs(i1 - i2) == abs(j1 - j2):
                clauses.append( [-self.varnum(i1, j1), -self.varnum(i2, j2)] )

        ## Solve
        assignment = solve(clauses)
        ans = []
        for i, j in product(range(self.N), repeat=2):
            if assignment[self.varnum(i, j) - 1] > 0:
                ans.append(j)
        print(ans)

In [17]:
%%time
# Solve N = 4
four_queens = NQueens(4)
four_queens.solve_board()

[2, 0, 3, 1]
CPU times: user 398 µs, sys: 24 µs, total: 422 µs
Wall time: 361 µs


In [18]:
%%time
# Solve N = 8
eight_queens = NQueens(8)
eight_queens.solve_board()

[7, 1, 4, 2, 0, 6, 3, 5]
CPU times: user 2.46 ms, sys: 30 µs, total: 2.49 ms
Wall time: 2.13 ms
